# L3 Data Visualization Notebook

This report comprises the entire Data Visualization of the L3 data (Level 3 Data)  of the Hotel Recommendation system built. 

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [5]:
# # Set up boto3 client (this step is optional and not necessary for using s3fs directly)
# s3_client = boto3.client('s3', region_name='us-west-2')

# # Define bucket and object key
# bucket_name = 'andorra-hotels-data-warehouse'
# object_key = 'l3_data/text/lsth.parquet'

# # Construct the S3 file path
# s3_file_path = f's3://{bucket_name}/{object_key}'

# Load the Parquet file into a pandas DataFrame
s3_file_path = "../output/l3_data_2024-09-01_10-02-26.parquet"
data = pd.read_parquet(s3_file_path, engine='pyarrow')

# Display the first few rows of the DataFrame
df_head = data.head()
df_head.style.hide()

avg_rating,user_ratings_total,review_rating,review_date_in_days,review_length,distance_to_ski_resort,distance_to_city_center,hotel_id,region_Andorra la Vella,region_Canillo,region_Encamp,region_Escaldes-Engordany,region_La Massana,region_Ordino,region_Sant Julià de Lòria,lang_ca,lang_en,lang_es,lang_fr,lang_other,sentiment,embedding,review_text_features
4.400000,581,5.000000,730,336,6.893862,9.107310,26,False,False,False,False,False,True,False,False,True,False,False,False,0.423571,[ 0.56704098 1.6374917 0.1200677 -1.02863395 -0.91445738 0.04996584 -0.84826899 -1.03662503 -0.18865408 -0.80020094 -0.20418474 -0.43226549 -0.46115026 0.73034334 -1.12254894 -0.69467801 -0.07586218 1.02359164 -0.28013745 -0.26449931 -0.35602066 0.69439501 -0.87602538 0.53709596 -0.75336611 -0.62288982 1.31949246 -0.22566174 0.84949666 0.44730726 -0.37368199 0.65003437 1.33353078 -0.29982978 -1.02994096 -0.24781904 0.68633413 -0.62884855 -0.37960893 0.45419958 -1.51452458 -0.44209793 -0.05089299 0.48445827 0.32774478 0.27114019 -0.87529725 0.59478754 0.55043793 -0.15677246 -0.98900402 0.2131895 0.20675278 0.63774019 0.20629726 -0.47785419 -0.86650169 0.70658886 -0.67992949 -0.32280827 0.18562803 1.74070477 -0.50180125 0.61090118 -0.26168373 -1.63699937 0.49376124 -0.1103339 -0.84229636 -0.37367499 0.63602829 -0.96289206 0.04961523 0.40385157 -0.75770009 1.22513461 0.21216528 0.26364511 0.44954696 -0.14268586 -0.13761115 0.62379342 0.38918564 0.21576998 -1.90831923 0.09099884 0.64842969 -0.64988422 0.11025935 1.56267035 0.97972739 0.36910659 0.72452247 -0.07581127 0.57665873 -1.1011517 -0.36628702 -1.67279112 -0.97459519 0.9338088 ],"{'type': 0, 'size': 20, 'indices': array([ 0, 1, 2, 3, 5, 6, 8, 9, 10, 12, 13, 16, 19], dtype=int32), 'values': array([0.69231034, 0.54698418, 1.80798955, 0.30003632, 1.71846211, 0.56908044, 1.08403249, 0.39137389, 1.35257876, 1.51427896, 0.74804554, 1.22680055, 0.72650068])}"
3.900000,1194,3.000000,60,216,4.926610,1.180006,131,False,False,True,False,False,False,False,False,False,False,False,True,0.196548,[ 0.6298241 1.13229203 0.42876887 -0.96355581 -1.20985365 0.18100145 -1.37215447 -0.96656686 -0.12404798 -0.63085693 -0.33176079 -0.61059225 -0.73874247 0.57635868 -1.6368643 -0.32615274 -0.03619017 1.42885327 -0.48722696 -0.0904194 -0.20970762 0.64635295 -0.91452086 0.85953343 -0.60463017 -0.52497351 1.4669416 -0.63045728 0.62448049 0.39433956 -0.55563319 0.71526426 1.55736971 -0.58027029 -1.04618561 -0.23319563 0.65170127 -0.85576695 0.08198832 0.20281172 -1.71152139 -0.29261842 -0.07575858 0.17673081 0.42516178 0.0831496 -0.85718471 0.35340986 0.87329113 -0.17055595 -1.10755897 -0.16376719 0.63513106 0.92340553 0.64438915 -0.24429269 -0.87216043 0.57962531 -0.63641542 -0.48537725 0.28854322 1.57705808 -0.31569818 0.71965158 0.14732191 -1.49787033 0.37466288 0.13817388 -0.68449795 -0.28007218 0.69937658 -0.77763051 0.28068516 -0.03934809 -0.54642367 1.17279601 0.19686568 -0.09691007 0.77510983 -0.26037538 -0.26424661 0.64220661 0.90417945 0.56360871 -1.66686308 0.36421686 0.66340983 -0.29834658 0.02716277 1.87535405 1.04492164 -0.01257665 0.93225956 -0.32783702 0.87125719 -0.72220695 -0.23065642 -1.57415438 -0.89035833 0.74372602],"{'type': 0, 'size': 20, 'indices': array([ 0, 2, 3, 4, 5, 6, 8, 9, 10, 14, 15, 16, 18], dtype=int32), 'values': array([0.69231034, 0.60266318, 0.60007264, 0.73495112, 1.1456414 , 0.56908044, 0.36134416, 0.39137389, 0.67628938, 0.54373105, 0.49237896, 1.22680055, 0.55726153])}"
4.200000,1081,4.000000,150,861,5.741960,1.934028,116,False,False,True,False,False,False,False,False,False,True,False,False,0.189063,[ 5.26986957e-01 9.32911873e-01 3.13869268e-01 -9.86761689e-01 -7.88426340e-01 4.25105214e-01 -1.42534232e+00 -1.28815770e+00 -8.35486650e-02 -5.72783947e-01 -5.31875789e-01 -3.49278986e-01 -6.12713873e-01 8.09343755e-01 -1.36659408e+00 -1.38365582e-01 9.34619382e-02 1.32793546e+00 -4.51055676e-01 -1.33468181e-01 -3.04768503e-01 7.76717424e-01 -5.76954663e-01 6.83365524e-01 -6.72984600e-01

In [7]:
# Exploratory Data Analysis
# Rating distribution
rating_fig = px.histogram(data, x='review_rating', nbins=5, title='Review Rating Distribution')

# Sentiment vs Review Length
sentiment_length_fig = px.scatter(data, x='review_length', y='sentiment', title='Sentiment vs Review Length')


In [8]:
# PCA or t-SNE Visualization of Embeddings
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
embedding_2d = pca.fit_transform(np.vstack(data['embedding']))

embedding_fig = px.scatter(embedding_2d, x=0, y=1, color=data['review_rating'], title='Embedding Visualization')


In [10]:
# Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Hotel Reviews Dashboard"),
    dcc.Dropdown(
        id='rating-filter',
        options=[{'label': str(i), 'value': i} for i in data['review_rating'].unique()],
        value=5,
        multi=True
    ),
    dcc.Graph(id='rating-distribution', figure=rating_fig),
    dcc.Graph(id='sentiment-length', figure=sentiment_length_fig),
    dcc.Graph(id='embedding-visualization', figure=embedding_fig),
])

@app.callback(
    Output('rating-distribution', 'figure'),
    Input('rating-filter', 'value')
)
def update_graph(selected_ratings):
    filtered_data = data[data['review_rating'].isin(selected_ratings)]
    return px.histogram(filtered_data, x='review_rating', nbins=5, title='Review Rating Distribution')

if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[10], line 22, in update_graph(selected_ratings=5)
     17 @app.callback(
     18     Output('rating-distribution', 'figure'),
     19     Input('rating-filter', 'value')
     20 )
     21 def update_graph(selected_ratings):
---> 22     filtered_data = data[data['review_rating'].isin(selected_ratings)]
        data =        avg_rating  user_ratings_total  review_rating  review_date_in_days  \
0             4.4                 581            5.0                  730   
1             3.9                1194            3.0                   60   
2             4.2                1081            4.0                  150   
3             3.8                 601            4.0                  730   
4             4.3                1257            4.0                  150   
...           ...                 ...            ...        

## 1. NLP: Review Text Features

The plot that visualizes the distribution of values for each feature extracted from the `review_text_features` column in the dataset. 
Each subplot represents the distribution of one of the features across all reviews, showing how frequently each value occurs.

In [ ]:
# Extract the review text features
def extract_review_text_features(row):
    try:
        features = row['review_text_features']
        return features['values']
    except (TypeError, KeyError):
        return []

df['review_text_values'] = df.apply(extract_review_text_features, axis=1)

# Create a DataFrame for review text features
review_text_df = pd.DataFrame(df['review_text_values'].tolist())

# Plot the distribution of each feature
if not review_text_df.empty:
    num_features = review_text_df.shape[1]
    num_cols = 4
    num_rows = (num_features // num_cols) + 1

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))

    # Flatten axes for easy iteration
    axes = axes.flatten()

    for i, column in enumerate(review_text_df.columns):
        sns.histplot(review_text_df[column], bins=20, kde=True, ax=axes[i])
        axes[i].set_title(f'Feature {i+1}')
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Frequency')

    # Remove empty subplots
    for j in range(i+1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()
else:
    print("No data available in review_text_df")